# Fully Connected Network

In [ ]:
import numpy as np
import pickle
import pandas as pd 
import matplotlib.pyplot as plt

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear, ReLU, GELU, Dropout

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
from torchinfo import summary

## Load data

In [ ]:
DATA_FOLDER = '../data'
PICKLE_FOLDER = '../pickles'

In [ ]:
with open(f'{PICKLE_FOLDER}/timeseries.pickle', 'rb') as f:
    ts = pickle.load(f)

In [ ]:
total_samples, total_brain_regions, ts_length = ts.shape

In [ ]:
df_metadata = pd.read_csv(f'{DATA_FOLDER}/patients-cleaned.csv', index_col=0)

In [ ]:
df_metadata.head(2)

### Select connectivity dataset

In [ ]:
CORR_TYPE = 'pearson'                                    # 'pearson', 'spearman', 'partial-pearson'

In [ ]:
fc_file = f'{PICKLE_FOLDER}/fc-{CORR_TYPE}.pickle'

In [ ]:
with open(fc_file, 'rb') as f:
    fc_matrix = pickle.load(f)

In [ ]:
fc_matrix.shape

## Split data

In [ ]:
with open(f'{PICKLE_FOLDER}/test-indices.pickle', 'rb') as f:
    test_indices = pickle.load(f)

In [ ]:
train_indices = list(set(range(total_samples)) - set(test_indices))

In [ ]:
train_targets = df_metadata.iloc[train_indices]["target"].reset_index(drop=True)

In [ ]:
print(f'Train set size: {len(train_indices)}')
print(f'Test set size: {len(test_indices)}')

## Prepare data

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
# Full flattened FC matrix.
def full_correlations_in_nodes(i):
    return torch.from_numpy(fc_matrix[i].reshape(-1)).to(torch.float32)

In [ ]:
# Flattened FC matrix, but disregard diagonal and lower triangle.
def triag_correlations_in_nodes(i):
    return torch.from_numpy(np.hstack([row[i+1:] for i, row in enumerate(fc_matrix[i])])).to(torch.float32)

In [ ]:
# Flattened FC matrix, but disregard diagonal and lower triangle.
def triag_abs_correlations_in_nodes(i):
    return torch.from_numpy(np.hstack([np.abs(row[i+1:]) for i, row in enumerate(fc_matrix[i])])).to(torch.float32)

In [ ]:
features_in_nodes = triag_abs_correlations_in_nodes
num_features_in_nodes = int(total_brain_regions**2 / 2 - 45)        # 8100 or 4005

### Create dataset

In [ ]:
dataset = [(
    features_in_nodes(i).to(device), 
    torch.tensor(target, dtype=torch.int64).to(device)
) for target, i in zip(train_targets, train_indices)]

In [ ]:
print(f'True train data: {len(dataset)}')

In [ ]:
print('Data object')
print(f'Features: {dataset[0][0].shape}')
print(f'Target: {dataset[0][1].shape}')

## Define model

In [ ]:
class FC(torch.nn.Module):
    
    def __init__(self, hidden_channels, p_dropout=0.5):
        super(FC, self).__init__()
        
        self.fc1 = Linear(num_features_in_nodes, hidden_channels)
        self.activation1 = GELU()
        self.dropout1 = Dropout()

        self.fc2 = Linear(hidden_channels, hidden_channels)
        self.activation2 = GELU()
        self.dropout2 = Dropout()

        self.fc3 = Linear(hidden_channels, hidden_channels)
        self.activation3 = GELU()
        self.dropout3 = Dropout()

        # self.fc4 = Linear(num_features_in_nodes+hidden_channels*3, 2)
        self.fc4 = Linear(hidden_channels, 2)

    def forward(self, x):

        # 1. Obtain node embeddings
        # 2. Remember layer outputs.
        out0 = x
        x = self.dropout1(self.activation1(self.fc1(x)))
        out1 = x
        x = self.dropout2(self.activation2(self.fc2(x)))
        out2 = x
        x = self.dropout3(self.activation3(self.fc3(x)))
        out3 = x

        # 3. Concatenate itermediate outputs.
        # x = torch.cat((out0, out1, out2, out3), 1)

        # 4. Fully connected for final classification.
        x = self.fc4(x)     # CELoss already incorporates `log_softmax`.
        
        return x

In [ ]:
summary(FC(8))

## Evaluation

In [ ]:
def evaluation_metrics(predicted, labels):
    pred_positives = predicted == 1
    label_positives = labels == 1

    tp = (pred_positives & label_positives).sum().item()
    tn = (~pred_positives & ~label_positives).sum().item()
    fp = (pred_positives & ~label_positives).sum().item()
    fn = (~pred_positives & label_positives).sum().item()

    return tp, tn, fp, fn

## Train model

In [ ]:
NUM_FOLDS = 3

In [ ]:
skf = StratifiedKFold(n_splits=NUM_FOLDS, random_state=42, shuffle=True)

In [ ]:
# Settings.
EPOCHS = 500
LR = 0.0001
MOMENTUM = 0.5
OPTIMIZER = 'adam'
LOSS = 'ce'
BATCH_SIZE = 16

VALIDATE_FREQ = 1

HIDDEN_CHANNELS = 16
DROPOUT = 0.5

STEP_SIZE = 50
GAMMA = 0.5

WEIGHT_DECAY = 0.0001

settings_str = f'bs={BATCH_SIZE},e={EPOCHS},lr={LR},mom={MOMENTUM},opt={OPTIMIZER},loss={LOSS},step={STEP_SIZE},gamma={GAMMA},wd={WEIGHT_DECAY},dropout={DROPOUT}'

In [ ]:
lr_lambda = lambda e: 0.01 if e < 200 else (0.001 if e < 300 else 0.0001)

In [ ]:
# Experiment folder.
EXP_FOLDER = 'runs/fc/triag/abs'

In [ ]:
# Experiment.
EXP_ID = 1

In [ ]:
for kfold, (train_index, val_index) in enumerate(skf.split(np.zeros(len(train_targets)), train_targets)):

    # Init TB writer.
    experiment_str = f'id={EXP_ID:03d},fold={kfold},{settings_str}'
    writer = SummaryWriter(f"../{EXP_FOLDER}/{experiment_str}")

    # Init model.
    net = FC(hidden_channels=HIDDEN_CHANNELS, p_dropout=DROPOUT).to(device)
    optimizr = torch.optim.Adam(net.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    # optimizr = torch.optim.SGD(net.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

    criterion = torch.nn.CrossEntropyLoss()
    #scheduler = torch.optim.lr_scheduler.StepLR(optimizr, step_size=STEP_SIZE, gamma=GAMMA)
    #scheduler = torch.optim.lr_scheduler.LambdaLR(optimizr, lr_lambda)

    # Save architecture.
    with open(f"../{EXP_FOLDER}/{experiment_str}/architecture", 'w', encoding="utf-8") as f:
        f.write(fc_folder + '\n')
        # f.write(fc_file_binary + '\n')
        f.write(fc_file_real + '\n')
        f.write(features_in_nodes.__str__() + '\n')
        f.write('\n'.join(experiment_str.split(',')) + '\n\n')
        f.write(net.__str__() + '\n\n')
        f.write(str(summary(net)))

    # Prepare data.
    X_train = [dataset[i] for i in train_index]
    X_val = [dataset[i] for i in val_index]
    
    trainloader = DataLoader(X_train, batch_size=BATCH_SIZE, shuffle=True)
    valloader = DataLoader(X_val, batch_size=BATCH_SIZE, shuffle=False)

    # Train.
    for epoch in range(EPOCHS):
        running_loss = 0.
        
        net.train()
        for x, y in trainloader:
            
            optimizr.zero_grad()
            outputs = net(x)   
            loss = criterion(outputs, y)  # Compute the loss.
            loss.backward()  # Derive gradients.
            optimizr.step()
            running_loss += loss.item()

        #scheduler.step()    # Update LR.
        epoch_loss = running_loss / len(trainloader)
        writer.add_scalar('training loss', epoch_loss, epoch)

        running_loss = 0.

        # Evaluate epoch.
        tp, tn, fp, fn = 0, 0, 0, 0
        total = 0

        net.eval()
        for x, y in valloader:

            optimizr.zero_grad()
            outputs = net(x) 
            loss = criterion(outputs, y)  # Compute the loss.
            running_loss += loss.item()

            if (epoch+1) % VALIDATE_FREQ == 0:
                predicted = outputs.argmax(dim=1)
                labels = y.view(-1)

                # Update.
                tp_, tn_, fp_, fn_ = evaluation_metrics(predicted, labels)
                tp += tp_; tn += tn_; fp += fp_; fn += fn_
                total += y.size(0)

        epoch_loss = running_loss / len(valloader)
        writer.add_scalar('validation loss', epoch_loss, epoch)
        
        if (epoch+1) % VALIDATE_FREQ == 0:
            writer.add_scalar('validation accuracy', (tp + tn) / total, epoch)
            writer.add_scalar('validation precision', tp / (tp + fp) if (tp + fp) > 0 else 0, epoch)
            writer.add_scalar('validation recall', tp / (tp + fn), epoch)
    
    # Single fold during exploration.
    #break

print('Finished training')